In [1]:
#instalar xlwt 
import pandas as pd
import numpy as np
import xlwt
from tkinter import filedialog
import tkinter as tk
from datetime import datetime

In [3]:
root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfile()

print("La carpeta seleccionada es:", file_path)


La carpeta seleccionada es: <_io.TextIOWrapper name='G:/Mi unidad/osoetsylra/importaciones_TIS_SD_ETC_a_sidefi/importacion_total_gecros_22-04-2023/total_afiliados_sidefi_2023-04-22.csv' mode='r' encoding='cp1252'>


#### Pasar a DATAFRAME

In [ ]:
padron = pd.read_csv(file_path, sep=";", encoding="ISO-8859-1")


#### calcular edades

In [23]:
def find_error(date_str):
    if not isinstance(date_str, str):
        date_str = str(date_str)
    if '"' in date_str:
        return date_str.find('"')
    else:
        return -1  # Devuelve -1 si no se encuentra el caracter problemático
padron['error_position'] = padron['Fecha Nacimiento'].apply(find_error)
registros_con_error = padron[padron['error_position'] != -1]

file_path = filedialog.asksaveasfilename(defaultextension='.csv')

# Si se seleccionó un archivo, guarda el dataframe en la ruta seleccionada
if file_path:
    if file_path.endswith('.csv'):
        registros_con_error.to_csv(file_path, sep = ";", encoding = "ISO-8859-1",index=False)
    elif file_path.endswith('.xlsx'):
        writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
        registros_con_error.to_excel(writer, sheet_name='Hoja1', index=False)
        writer.save()
    else:
        print('Extensión de archivo no válida.')


### corregir fechas

In [24]:

#fecha de nacimiento calcular edad
def calcular_edad(fecha_nacimiento):
    if pd.isnull(fecha_nacimiento):
        return None
    try:
        fecha_nacimiento = datetime.strptime(str(fecha_nacimiento), "%d/%m/%Y")
        edad = int((datetime.now() - fecha_nacimiento).days / 365.25)
        return edad
    except ValueError:
        return "Error: formato de fecha inválido"

# Ejemplo de uso
padron['edad'] = padron['Fecha Nacimiento'].apply(calcular_edad)
print(padron.head(7))
edades = padron[['edad','Fecha Nacimiento','Apellido Afiliado','Nombre Afiliado','Nro. Documento']]

file_path = filedialog.asksaveasfilename(defaultextension='.csv')

# Si se seleccionó un archivo, guarda el dataframe en la ruta seleccionada
if file_path:
    if file_path.endswith('.csv'):
        edades.to_csv(file_path, sep = ";", encoding = "cp1252",index=False)
    elif file_path.endswith('.xlsx'):
        writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
        edades.to_excel(writer, sheet_name='Hoja1', index=False)
        writer.save()
    else:
        print('Extensión de archivo no válida.')





   Nro. OS Plan del Afiliado  Nro. Grupo Familiar    Nro. Afiliado  \
0      3.0          Afiliado           30851327.0              NaN   
1      6.0          Afiliado           30694069.0  010-30694069-00   
2      1.0          Afiliado           32479768.0              NaN   
3      1.0          Afiliado           10961767.0              NaN   
4      1.0          Afiliado           30034293.0              NaN   
5      1.0          Afiliado           25084989.0              NaN   
6      1.0          Afiliado           21981341.0              NaN   

  Parentesco con el TITULAR Apellido Afiliado    Nombre Afiliado  \
0                   TITULAR             ABACA            MARCELO   
1                   TITULAR             ABADI         PEDRO RAUL   
2                   TITULAR            ABADIE      BRENDA ROXANA   
3                   TITULAR            ABADIE      MONICA ISABEL   
4                   TITULAR              ABAL     NATALIA ANDREA   
5                   TITULAR    

### mostrar duplicados

In [ ]:
#duplicados exportar
duplicados = padron[padron.duplicated(subset=['Nro. Documento'], keep=False)]
print("Los valores duplicados en la Nro. Documento son:")
duplicadodf = (duplicados['Nro. Documento'].unique())

duplicados.to_csv('C:\padrones\duplicados.csv', sep = ";", encoding="ISO-8859-1", index=False)

### Transformacion del padron para Gecros

In [5]:
padron = pd.read_csv(file_path, sep=";", encoding="ISO-8859-1")
#delete col 'opcion' not required for gecros format. 
padron = padron.drop('Opcion', axis = 1)
#replace all columns that generate errors on import staging
padron['Departamento'] = [''] * len(padron)
padron['Telefono'] = [''] * len(padron)
padron['Celular'] = [''] * len(padron)
padron['Codigo Postal'] = [''] * len(padron)
#replace columns 'Tipo Documento' equivalences ---> DNI, CI, LE
diccionario_dni = {'Documento Nacional Identidad': 'DNI', 'Cedula Identidad': 'CI','00-00-0000': '01-01-2020'}
padron[['Tipo Documento','Fecha Alta']] = padron[['Tipo Documento','Fecha Alta']].replace(diccionario_dni)
#print(padron.head(5))

In [9]:
padron1 = padron[padron['Nro. OS'] == 1]
padron1.to_excel('C:\padrones\osoetsylra.xls',index = False )
#padron1.to_csv('C:\padrones\osoetsylra.csv', index = False, sep = '|' )

C:\Users\zickd\AppData\Local\Temp\ipykernel_4304\3499769349.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  padron1.to_excel('C:\padrones\osoetsylra.xls',index = False )


In [10]:
padron2 = padron[padron['Nro. OS'] == 2]
padron2.to_excel('C:\padrones\otros.xls', engine='xlwt' ,index = False )

C:\Users\zickd\AppData\Local\Temp\ipykernel_4304\2629869373.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  padron2.to_excel('C:\padrones\otros.xls', engine='xlwt' ,index = False )


In [11]:
padron3 = padron[padron['Nro. OS'] == 3]
padron3.to_excel('C:\padrones\pvisitar.xls', engine='xlwt' ,index = False )

C:\Users\zickd\AppData\Local\Temp\ipykernel_4304\3434180392.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  padron3.to_excel('C:\padrones\pvisitar.xls', engine='xlwt' ,index = False )


In [12]:
padron4 = padron[padron['Nro. OS'] == 4]
padron4.to_excel('C:\padrones\sd.xls', index = False )
# workbook and sheet
#book4 = xlwt.Workbook(encoding="utf-8")
#sheet = book4.add_sheet("sd")
# Escribimos los encabezados de columna

#for i, header in enumerate(padron4.columns):
 #   sheet.write(0, i, header)
# Escribimos los datos en la hoja de trabajo
#for r, row in enumerate(padron4.values):
 #   for c, val in enumerate(row):
  #      sheet.write(r+1, c, val)
#book4.save('C:\padrones\sd.xls')

C:\Users\zickd\AppData\Local\Temp\ipykernel_4304\1691951161.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  padron4.to_excel('C:\padrones\sd.xls', index = False )


In [13]:
padron5 = padron[padron['Nro. OS'] == 5]
padron5.to_excel('C:\padrones\ptis.xls', index = False )

C:\Users\zickd\AppData\Local\Temp\ipykernel_4304\3980072344.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  padron5.to_excel('C:\padrones\ptis.xls', index = False )


In [14]:
padron6 = padron[padron['Nro. OS'] == 6]
padron6.to_excel('C:\padrones\gerensal.xls', index = False )

C:\Users\zickd\AppData\Local\Temp\ipykernel_4304\3816216975.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  padron6.to_excel('C:\padrones\gerensal.xls', index = False )
